In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import celerite
from celerite import terms

In [ ]:
### TESS data import ###
# data = np.loadtxt("/home/marc/Stages/Stage_M2/TESS/tessbpic.dat",usecols=(0,1))
# time = data[:,0]
# #time = np.reshape(time,[32,int(62976/32)])
# flux = data[:,1]
# #flux = np.reshape(flux,[32,int(62976/32)])

# mean = np.mean(flux)
# flux-=mean

# norm = np.max(flux)
# flux/=norm

# day1 = time[0]
# time-=day1

time = np.sort(np.append(
    np.random.uniform(0, 3.8, 5700),
    np.random.uniform(5.5, 10, 6800),
))  # The input coordinates must be sorted
flux_error = np.random.uniform(0.08, 0.22, len(t))
flux = np.sin(3*t + 0.1*(t-5)**2) + flux_error * np.random.randn(len(t))

In [ ]:
### Uncertities ###
# stock_flux = []
# for i in range(500) :
#     tirage = np.random.randint(0,len(flux),len(flux))
#     stock_flux.append(np.mean(flux[tirage]))
# flux_error = np.var(flux)
#flux_error = np.random.uniform(1e-3,1e-2,len(time))


In [ ]:
### Data plot ###
fig = plt.figure()
ax = plt.gca()
#ax.scatter(time,flux,c="r",s=1)
ax.errorbar(time,flux,yerr=flux_error,fmt='.r',markersize=1,capsize=0)
ax.set_xlabel("Time")
ax.set_ylabel("Flux")
fig.show()

In [ ]:
### GP Model ###
Q = 1./np.sqrt(2)
w0 = 1.
S0 = np.var(flux)
kernel = terms.SHOTerm(log_S0=np.log(S0), log_Q=np.log(Q), log_omega0=np.log(w0))

gp = celerite.GP(kernel)
gp.compute(time,flux_error) # You always need to call compute once.
print("Initial log likelihood: {0}".format(gp.log_likelihood(flux)))
print("parameter_dict:\n{0}\n".format(gp.get_parameter_dict()))

In [ ]:
### Optimisation ###
from scipy.optimize import minimize

def neg_log_like(params, flux, gp):
    gp.set_parameter_vector(params)
    return -gp.log_likelihood(flux)

initial_params = gp.get_parameter_vector()

results = minimize(neg_log_like, initial_params, method="L-BFGS-B", args=(flux, gp))
print(results)

In [ ]:
x = np.linspace(np.min(time),np.max(time),20000)
pred_mean, pred_var = gp.predict(flux, x,return_var=True)
pred_std = np.sqrt(pred_var)

In [ ]:
color = "#ff7f0e"
plt.errorbar(time,flux,yerr=flux_error,fmt=".r",capsize=0,markersize=1)
plt.plot(x, pred_mean, color=color)
plt.fill_between(x, pred_mean+pred_std, pred_mean-pred_std, color=color, alpha=0.3, edgecolor="none")
plt.xlabel("x")
plt.ylabel("y")
plt.show()